<a href="https://colab.research.google.com/github/Armestrong/deploy_preco_imoveis_sp/blob/master/Deploy_para_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preço de Imóveis em São Paulo

Neste módulo, iremos treinar um modelo para fazer a previsão do preço de venda de apartamentos na cidade de São Paulo e usar esse modelo para alimentar uma aplicação web mediante *deploy*.

Como o objetivo é focar na construção do *webapp* e em como subir uma aplicação, a etapa da análise exploratória será suprimida (feita anteriormente por mim).

Como identifiquei as colunas desnecessárias e redundantes, irei direto ao ponto do treinamento, visando mostrar principalmente como exportar e importar o modelo com a biblioteca `pickle`.

## Dados de Imóveis

Os dados usados aqui foram obtidos [neste link](https://www.kaggle.com/argonalyst/sao-paulo-real-estate-sale-rent-april-2019), e foram disponibilizados publicamente pela startup OpenImob.

Quero ressaltar aqui que este banco de dados contém poucas *features* comparado com tudo que seria possível, além de só representar os imóveis para o mês de abril de 2019.

Para facilitar seu projeto, disponibilizei o arquivo `csv` [neste link](https://www.dropbox.com/s/h8blgaphkfpqsn5/sao-paulo-properties-april-2019.csv?dl=1), a partir do meu Dropbox.

## Análise e Tratamento dos Dados

Os dados originais contém 13.640 entradas e 16 colunas, sendo a coluna `Price` a nossa variável alvo.

In [0]:
# importar os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# importar o dataset para um dataframe
url_dataset = "https://www.dropbox.com/s/h8blgaphkfpqsn5/sao-paulo-properties-april-2019.csv?dl=1"
df = pd.read_csv(url_dataset)

# ver as 5 primeiras entradas
display(df.head())

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


Para fazer a limpeza, clonei o *dataframe* original e eliminei as colunas `['Property Type', 'Latitude', 'Longitude']`.

Se você também reparar acima, os nomes dos bairros tinham uma informação desnecessária para este *dataset* específico, acrescentando a *string* `"/São Paulo"` ao final de cara nome. Usando `df_clean['District'].apply(lambda x: x.split('/')[0]` eu simplesmente removi essa informação e deixei mais limpa a coluna.

Se você explorar melhor esse *dataset* vai ver que ele contempla duas situações: aluguel ou venda. Neste projeto, iremos trabalhar exclusivamente com os casos de venda, ou seja, apenas quando `df_clean['Negotiation Type'] == 'sale'`.

In [0]:
lista_bairros

In [0]:
df_clean = df.copy()

# Limpar os nomes do bairros
df_clean['District'] = (df_clean['District'].apply(lambda x: x.split('/')[0]))

# excluir as colunas 'Latitude', 'Longitude' e 'Property Type'
df_clean.drop(['Property Type', 'Latitude', 'Longitude'], axis=1, inplace=True)

# extrair apenas as entradas onde df_clean['Negotiation Type'] == 'sale'
df_sale = df_clean[df_clean['Negotiation Type'] == 'sale'].copy()
df_sale.drop(['Negotiation Type'], axis=1, inplace=True)

# ver as 5 primeiras entradas
df_sale.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District
4901,732600,1000,74,1,2,1,2,1,0,1,0,Vila Madalena
4902,1990000,2400,164,4,5,2,3,1,1,1,0,Vila Madalena
4903,720000,700,70,2,2,1,1,1,0,1,1,Vila Madalena
4904,1680000,1580,155,3,5,3,2,1,0,1,0,Vila Madalena
4905,1200000,900,56,2,2,1,2,0,1,1,0,Vila Madalena


In [0]:
lista_bairros = df_clean['District'].apply(lambda x: x.split('/')[0])
lista_bairros = sorted(lista_bairros.unique())


## Modelo de Machine Learning

### Versão Simplificada

Seguindo o que foi explicado nas aulas e visando **focar no processo** (e não em um projeto específicamente), vou criar um modelo simplificado de *machine learning*, que usará apenas as 4 *features* numéricas `['Condo', 'Size', 'Rooms', 'Suites']`.

Arbitrariamente, escolhi o modelo XGBoost para treinar meu modelo e observei três principais métricas de avaliação - apesar de saber das limitações do *dataset* e do modelo simplificado.

In [0]:
# metodo de conversao 1
df_sale.District = df_sale.District.astype('category')
df_sale.District = df_sale.District.cat.codes


In [0]:
##### encoding coluna distrito ###
labelencoder = LabelEncoder()
df_sale.District = labelencoder.fit_transform(df_sale.District)


In [0]:

# separar entre variáveis X e y
X_simp = df_sale[['Condo', 'Size', 'Rooms','Toilets', 'Suites','Parking','Elevator','Furnished','Swimming Pool','New','District']]
y_simp = df_sale['Price']

#############################
data_dmatrix = xgb.DMatrix(data=X_simp,label=y_simp)

# split entre datasets de treino e teste
X_train_simp, X_test_simp, y_train_simp, y_test_simp = train_test_split(X_simp, y_simp, test_size=0.33)

# instanciar e treinar o modelo
model_simp = XGBRegressor(random_state=42, silent=True)
model_simp.fit( X_train_simp, y_train_simp)


# fazer as previsões em cima do dataset de teste
y_pred_simp = model_simp.predict(X_test_simp)

# métricas de avaliação
print("r2: \t{:.4f}".format(r2_score(y_test_simp, y_pred_simp)))
print("MAE: \t{:.4f}".format(mean_absolute_error(y_test_simp, y_pred_simp)))
print("MSE: \t{:.4f}".format(mean_squared_error(y_test_simp, y_pred_simp)))

r2: 	0.8091
MAE: 	141040.5923
MSE: 	91221798887.4329


In [0]:
model_simp

#### Salvando o modelo

O nosso modelo está treinado e é capaz de realizar previsões. No entanto, está "preso" ao *kernel* rodando dentro do Google Colab.

Imagine precisar rodar todas as células novamente a cada vez que fosse fazer uma previsão. Seria inviável!

Para conseguir exportar o modelo de *machine learning* (na verdade, isso pode ser feito com qualquer estrutura de dados) vou usar a biblioteca `pickle`. Abaixo, eu salvo dentro do arquivo `modelo_simples.pkl` o modelo treinado que foi atribuido à variável `model_simp`.

obs: O arquivo `modelo_simples.pkl` precisa ser salvo em `modelo_simples.json` pois se vc estiver usando sua maquina virtual tipo eu que uso Visual Studio, ele requer que vc user `.json` caso o contrario, o arquivo não renderizado.

https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

In [0]:
display(X_train_simp)

,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District
12375,0,110,3,2,1,2,0,0,1,0,85
12429,380,58,3,2,1,1,0,0,1,0,50
6243,519,55,2,2,1,1,0,0,1,0,27
7222,293,60,2,2,1,1,0,0,0,0,42
7574,1512,90,3,3,1,1,1,0,0,0,90
...,...,...,...,...,...,...,...,...,...,...,...
7637,0,266,4,5,4,4,1,0,0,0,1
7262,360,52,2,2,1,1,0,0,0,0,41
6161,400,68,2,2,1,2,1,0,1,1,19
12622,400,49,2,1,0,1,1,0,0,0,17


In [0]:
# salvar o modelo em formato pkl
import pickle

with open('modelo_simples.pkl', 'wb') as file:
    pickle.dump(model_simp, file)



Uma vez que você exporta o seu modelo, é extremamente importante que você também salve os nomes das *features* que esse modelo espera receber, e tem que ser na ordem exata que ele foi treinado.

Da mesma maneira que fizemos com o modelo, salvei os nomes das variáveis em `features_simples.names`.

In [0]:
#Salvar numeros dos bairros
lista_nr_bairros = labelencoder.fit_transform(lista_bairros)

In [0]:
# arquivo de nomes de bairros & numeros em seguencia
with open('lista_bairros.names', 'wb') as file:
  pickle.dump(lista_bairros, file)
with open('lista_bairros_nr.names', 'wb') as file:
  pickle.dump(lista_nr_bairros, file)

In [0]:
# salvar os nomes das features do modelo simples
features_simples = X_train_simp.columns.values
with open('features_simples.names', 'wb') as file:
    pickle.dump(features_simples, file)


In [0]:
features_simples

array(['Condo', 'Size', 'Rooms', 'Toilets', 'Suites', 'Parking',
       'Elevator', 'Furnished', 'Swimming Pool', 'New', 'District'],
      dtype=object)

#### Carregando o modelo

Uma vez que você salvou o modelo em um arquivo, consegue carregar ele novamente usando o `pickle.load()`

In [0]:
# importar modelo e feature names
with open('/content/modelo_simples.pkl', 'rb') as file:
    modelo_simples = pickle.load(file)
with open('/content/features_simples.names', 'rb') as file:
    features_names = pickle.load(file)
with open('/content/lista_bairros.names', 'rb') as file:
    lista_bairros = pickle.load(file)
with open('/content/lista_bairros_nr.names', 'rb') as file:
    lista_nr_bairros = pickle.load(file)

In [0]:
# ver o tipo da nova variável
type(modelo_simples)

xgboost.sklearn.XGBRegressor

#### Transformando inputs em Dataframes

Você já consegue extrair os valores, mas para poder passar esses valores no modelo é necessário transformá-los em um Dataframe.


In [0]:
# input do usuário
user_inputs = {
    'Condo':"123", 'Size':"122", 'Rooms':"1", 'Toilets':"1", 'Suites':"1", 'Parking':"1",
       'Elevator':"0", 'Furnished':"1", 'Swimming Pool':"1", 'New':"1", 'District':"1"
}

In [0]:
# input para dataframe (em branco)
df = pd.DataFrame(index=[0], columns=features_names)
df = df.fillna(value=0)

for i in user_inputs.items():
    df[i[0]] = i[1]
    
df = df.astype(float)

Agora é só fazer a previsão com o novo Dataframe.

In [0]:
y_pred = modelo_simples.predict(df)[0]

In [0]:
print(features_simples)

['Condo' 'Size' 'Rooms' 'Toilets' 'Suites' 'Parking' 'Elevator'
 'Furnished' 'Swimming Pool' 'New' 'District']


In [0]:
lista_nr_bairros

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95])

In [0]:
#PROVAVELMENTE PARA ARRUMAR O ERRO QUE DA NA NOVA VERSAO DO XGBOOST

In [0]:
model_simp.save_model('mlo.pkl')

In [0]:
# instanciar novamente o modelo
ml = XGBRegressor()
ml.load_model('/content/mlo.pkl')